# 🧠 QOSF Monthly Challenge: Working with NISQ Devices (Draper Adder + Mitiq)


Welcome to the Quantum Open Source Foundation (QOSF) Monthly Challenge!
In this notebook, we’ll walk through building a **Draper Adder** using the **Quantum Fourier Transform (QFT)** and learn to mitigate noise using the Mitiq library's **Zero-Noise Extrapolation (ZNE)** technique.

You’ll learn:
- How to encode integers into quantum states
- How QFT allows for arithmetic on quantum registers
- How real-world quantum noise affects computation
- How to apply error mitigation using Mitiq

References:
- QFT: https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html
- Draper Adder: https://arxiv.org/abs/quant-ph/0008033
- Mitiq Docs: https://mitiq.readthedocs.io/


# 🎯 Objectives:
- Implement a `draper_adder(n, n1, n2)` quantum circuit
- Simulate the circuit in both **ideal** and **noisy** conditions
- Apply **Zero-Noise Extrapolation (ZNE)** using Mitiq
- Optimize your implementation and compare results

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error
import numpy as np
import matplotlib.pyplot as plt
from mitiq import zne
from mitiq.zne.scaling import fold_global
from mitiq.zne.inference import LinearFactory, ExpFactory, RichardsonFactory, PolyFactory
from mitiq.interface.mitiq_qiskit import from_qiskit, to_qiskit

# 🧠 Section 1: QFT and Inverse QFT

Quantum Fourier Transform (QFT) is a core building block in many quantum algorithms, and it enables addition using phase rotations. In this section, we define QFT and its inverse.

In [ ]:
def qft(circuit, n):
    for i in range(n):
        circuit.h(i)
        for j in range(i + 1, n):
            angle = np.pi / (2 ** (j - i))
            circuit.cp(angle, j, i)
    circuit.barrier()

def inverse_qft(circuit, n):
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            angle = -np.pi / (2 ** (j - i))
            circuit.cp(angle, j, i)
        circuit.h(i)
    circuit.barrier()

# 🏗️ Section 2: Draper Adder Circuit

We construct the Draper Adder circuit. This adds two integers using QFT.
Input: Integers n1 and n2 in `n` qubits
Output: Quantum state representing n1 + n2


In [ ]:
def draper_adder(n, n1, n2):
    qc = QuantumCircuit(n)
    for i in range(n):
        if (n1 >> i) & 1:
            qc.x(i)
    qc.barrier()

    qft(qc, n)

    for i in range(n):
        for j in range(n):
            if (n2 >> j) & 1:
                angle = np.pi / (2 ** (i - j)) if i >= j else 0
                if angle:
                    qc.cp(angle, j, i)
    qc.barrier()

    inverse_qft(qc, n)
    qc.measure_all()
    return qc

# 🧪 Section 3: Ideal Simulation

We simulate the circuit with no noise using Qiskit Aer.
Expected result: 5 + 2 = 7 → '111' on 3 qubits.

In [ ]:
qc = draper_adder(3, 5, 2)
sim = Aer.get_backend("aer_simulator")
t_qc = transpile(qc, sim)
result = sim.run(assemble(t_qc)).result()
counts = result.get_counts()
print("\n✅ Ideal Result (no noise):", counts)
plot_histogram(counts)
plt.show()

# 🤖 Section 4: Noisy Simulation

To mimic real quantum devices, we add 2% depolarizing noise to X, H, and CP gates. Observe the degradation of the expected result.

In [ ]:
noise_model = NoiseModel()
error = depolarizing_error(0.02, 1)
noise_model.add_all_qubit_quantum_error(error, ["x", "h", "cp"])

noisy_result = sim.run(assemble(t_qc), noise_model=noise_model).result()
noisy_counts = noisy_result.get_counts()
print("\n🔧 Noisy Result (with 2% depolarizing noise):", noisy_counts)
plot_histogram(noisy_counts)
plt.show()

# 🧰 Section 5: Error Mitigation using Mitiq (ZNE)

Zero-Noise Extrapolation (ZNE) estimates ideal outcomes by executing circuits at multiple noise levels.
We use four factory methods to compare error mitigation effectiveness.

In [ ]:
from qiskit import ClassicalRegister

def executor(circuit):
    circuit.measure_all()
    t_circ = transpile(to_qiskit(circuit), sim)
    qobj = assemble(t_circ, shots=1024)
    result = sim.run(qobj, noise_model=noise_model).result()
    counts = result.get_counts()
    return counts.get("111", 0) / 1024

mitiq_circuit = from_qiskit(qc.remove_final_measurements(inplace=False))
scale_factors = [1.0, 2.0, 3.0]

factories = {
    "Linear": LinearFactory(scale_factors=scale_factors),
    "Exponential": ExpFactory(scale_factors=scale_factors, asymptote=0.0),
    "Richardson": RichardsonFactory(scale_factors=scale_factors),
    "Polynomial": PolyFactory(scale_factors=scale_factors, order=2),
}

results = {}
for name, factory in factories.items():
    results[name] = zne.execute_with_zne(
        mitiq_circuit, executor, factory=factory, scale_noise=fold_global
    )
    print(f"📈 {name} ZNE result:", results[name])

# 📊 Section 6: Summary of Results

In [ ]:
print("\n📊 Summary of Results:")
print("Unmitigated:", executor(mitiq_circuit))
for name, value in results.items():
    print(f"{name}: {value}")

# 🔍 Section 7: Testing multiple 3-bit inputs

This section systematically tests all possible 3-bit integer additions and compares quantum output to classical.

In [ ]:
def classical_adder(n, n1, n2):
    total = (n1 + n2) % (2**n)
    return format(total, f"0{n}b")

def test_draper_adder(n=3):
    for n1 in range(2**n):
        for n2 in range(2**n):
            qc = draper_adder(n, n1, n2)
            result = sim.run(assemble(transpile(qc, sim))).result()
            counts = result.get_counts()
            expected = classical_adder(n, n1, n2)
            top = max(counts, key=counts.get)
            status = "✅" if top == expected else "❌"
            print(f"{n1}+{n2} → {top} (expected: {expected}) {status}")

test_draper_adder()

# 📉 Section 8: Analyze Noise Level Effects

We now analyze how different depolarizing noise levels affect accuracy of the output.

In [ ]:
noise_levels = np.linspace(0, 0.1, 10)
accuracies = []

for level in noise_levels:
    temp_model = NoiseModel()
    temp_model.add_all_qubit_quantum_error(depolarizing_error(level, 1), ["x", "h", "cp"])
    result = sim.run(assemble(t_qc), noise_model=temp_model).result()
    counts = result.get_counts()
    accuracies.append(counts.get("111", 0) / 1024)

plt.plot(noise_levels, accuracies, marker='o')
plt.title("Accuracy vs Depolarizing Noise Level")
plt.xlabel("Depolarizing Error Rate")
plt.ylabel("Probability of Correct Result")
plt.grid(True)
plt.show()

# ⚙️ Section 9: Bonus Optimization (Optional TODO)

Optional: Try optimizing your Draper Adder implementation to reduce circuit depth and improve resilience.


In [ ]:
# TODO: Try implementing an optimized version of Draper Adder
def draper_adder_v2(n, n1, n2):
    ...

# 🏁 Section 10: Conclusion

In this challenge, we built a Draper adder—a quantum circuit that adds numbers using the QFT.
We observed ideal vs noisy simulation and applied error mitigation via Mitiq’s ZNE.
We also evaluated different ZNE strategies and performed broad tests and noise analysis.

You're now equipped with insights into quantum arithmetic and NISQ error mitigation!


# ✅ Submission Instructions:
 - Submit your notebook to: https://github.com/qosf/monthly-challenges
 - Include your name and GitHub handle
 - Ask a peer to review your PR on Slack

**🎉 Good luck!**
